Sequential Chats and Customer Onboarding

## Setup

In [1]:
llm_config = {"model": "llama3.1","api_type":"ollama"}

In [2]:
from autogen import ConversableAgent

## Creating the needed agents

In [3]:


from autogen import ConversableAgent

onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",  # Removed spaces
    system_message='''You are a helpful customer onboarding agent.
    You are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",  # Removed spaces
    system_message='''You are a helpful customer service agent.
    You are here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)


## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [4]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": "Hello, I'm here to help you get started with our product. Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as JSON object only: {'name': '', 'location': ''}",
        },
        "max_turns": 1,
        "clear_history": True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Great! Could you tell me what topics you are interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [5]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)



********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product. Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\Users\Pandiyan\anaconda3\envs\agentai\Lib\site-packages\autogen\agentchat\chat.py:57: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Pandi from Sivakasi


customer_proxy_agent (to onboarding_personal_information_agent):

Pandi from Sivakasi

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (b5e08d93-ccfc-4629-b7cd-f5b557a5789f): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_topic_preference_agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{'name': 'Pandi', 'location': 'Sivakasi'}

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to onboarding_topic_preference_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Gospels from Bible


customer_proxy_agent (to onboarding_topic_preference_agent):

Gospels from Bible

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (f7446a04-875c-4c1a-9af1-a1e933933c40): Maximum turns (1) reached

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to customer_engagement_agent):

Let's find something fun to read.
Context: 
{'name': 'Pandi', 'location': 'Sivakasi'}
The person, Pandi from Sivakasi, is interested in reading about Gospels from the Bible.

--------------------------------------------------------------------------------
customer_engagement_agent (to customer_proxy_agent):

Namaste! Hello Pandi!

I've got a treat for you! As someone who's fascinated by the Gospels of the Bible, I'd like to share an interesting fact with you.

Did you know that Sivakasi, your homet

## Print out the summary

In [6]:
summary=[]
for chat_result in chat_results:
    print(chat_result.summary)
    summary.append(chat_result.summary)
    print("\n")

{'content': "{'name': 'Pandi', 'location': 'Sivakasi'}", 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': 'The person, Pandi from Sivakasi, is interested in reading about Gospels from the Bible.', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


{'content': "Pandi from Sivakasi is interested in reading about Gospels from the Bible.\nA connection was made between Pandi's hometown, known as the 'Printing City', and its history of printing and publishing religious texts, including Bibles.\nBartholomew, an apostle of Jesus, is said to have preached in India according to tradition.", 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}




In [7]:
summary[0].get("content")

"{'name': 'Pandi', 'location': 'Sivakasi'}"

In [ ]:
#!pip install pandas

In [8]:
import pandas as pd
import ast

# Input string
#data_str = "{'name': 'Ramisha', 'location': 'Coimbatore'}"
data_str=summary[0].get("content")
# Convert string to dictionary
data_dict = ast.literal_eval(data_str)

# Convert dictionary to DataFrame
df = pd.DataFrame([data_dict])

df


name  location
0  Pandi  Sivakasi

In [9]:
summary[1]

{'content': 'The person, Pandi from Sivakasi, is interested in reading about Gospels from the Bible.',
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [10]:
df["TopicPreference"]=summary[1].get("content")

In [11]:
df

name  location                                    TopicPreference
0  Pandi  Sivakasi  The person, Pandi from Sivakasi, is interested...

In [12]:
summary[2]

{'content': "Pandi from Sivakasi is interested in reading about Gospels from the Bible.\nA connection was made between Pandi's hometown, known as the 'Printing City', and its history of printing and publishing religious texts, including Bibles.\nBartholomew, an apostle of Jesus, is said to have preached in India according to tradition.",
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [13]:
chat_results[2].chat_history[1].get("content")

"Namaste! Hello Pandi!\n\nI've got a treat for you! As someone who's fascinated by the Gospels of the Bible, I'd like to share an interesting fact with you.\n\nDid you know that Sivakasi, your hometown, is known as the 'Printing City' in India? It's not just because of its bustling print industry, but also because the city has a long history of printing and publishing religious texts, including Bibles!\n\nNow, let me share a fun fact about one of Jesus' apostles. Did you know that Bartholomew, who was a close companion of Jesus, is said to have preached in India? Yes, you read that right! According to tradition, he traveled to the subcontinent and shared the Gospel with many people.\n\nIf you're interested in reading more about Gospels or would like some book recommendations related to your hometown's connection to Christianity, I'd be happy to help!\n\nWhat do you say? Would you like me to share another fun fact or recommend a book on this topic?\n\n(And by the way, have you ever hear

In [14]:
df["Enagement"]=chat_results[2].chat_history[1].get("content")

In [15]:
df

name  location                                    TopicPreference  \
0  Pandi  Sivakasi  The person, Pandi from Sivakasi, is interested...   

                                           Enagement  
0  Namaste! Hello Pandi!\n\nI've got a treat for ...

In [ ]:
#!pip install gspread
#!pip install oauth2client

In [16]:
def add_leads_to_google_sheet(df):
    import gspread
    from oauth2client.service_account import ServiceAccountCredentials

    # Define the scope for Google Sheets API
    scope = ['https://www.googleapis.com/auth/spreadsheets']

    # Add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name(
        'fair-subset-455717-s6-84dee767824c.json', scope
    )

    # Authorize the client
    client = gspread.authorize(creds)

    # Specify the Google Sheet ID
    sheet_id = '1gLkaCmWgQp_fuEqUeOkf-SsjwYfh4PuFQ2tpEKoLTFI'

    # Open the spreadsheet using its ID
    spreadsheet = client.open_by_key(sheet_id)

    # Select the first worksheet (index 0)
    worksheet = spreadsheet.get_worksheet(0)

    # Append each row of the DataFrame to the worksheet
    for _, row in df.iterrows():
        # Convert row to a list and append to the sheet
        worksheet.append_row(row.tolist())

    print("Data added successfully!")
    return "added"


In [17]:
add_leads_to_google_sheet(df)

Data added successfully!


'added'

## Print out the cost

In [18]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0, 'llama3.1': {'cost': 0.0, 'prompt_tokens': 68, 'completion_tokens': 17, 'total_tokens': 85}}, 'usage_excluding_cached_inference': {'total_cost': 0.0, 'llama3.1': {'cost': 0.0, 'prompt_tokens': 68, 'completion_tokens': 17, 'total_tokens': 85}}}


{'usage_including_cached_inference': {'total_cost': 0.0, 'llama3.1': {'cost': 0.0, 'prompt_tokens': 74, 'completion_tokens': 23, 'total_tokens': 97}}, 'usage_excluding_cached_inference': {'total_cost': 0.0, 'llama3.1': {'cost': 0.0, 'prompt_tokens': 74, 'completion_tokens': 23, 'total_tokens': 97}}}


{'usage_including_cached_inference': {'total_cost': 0.0, 'llama3.1': {'cost': 0.0, 'prompt_tokens': 464, 'completion_tokens': 323, 'total_tokens': 787}}, 'usage_excluding_cached_inference': {'total_cost': 0.0, 'llama3.1': {'cost': 0.0, 'prompt_tokens': 464, 'completion_tokens': 323, 'total_tokens': 787}}}


